# Práctica 3. Bolsas de palabras
Guillermo Segura Gómez

Vamos a utilizar un corpus llamado [MEX-A3T](https://sites.google.com/view/mex-a3t/home?authuser=0) que es un corpus creado en México. MEX-A3T es un corpus hecho para poder medir la agresividad y las fake news. El equipo de CIMAT se dedicó al análisis de identificación de agresividad. 

Lo primero que es necesario hacer es leer los datos del corpus. El corpus incluye datos de entrenamiento asi como etiquetas.

**Keras**

Se introduce la libería **keras**. Keras es una biblioteca de Redes Neuronales de Código Abierto escrita en Python. Es capaz de ejecutarse sobre TensorFlow, Microsoft Cognitive Toolkit o Theano.

Está especialmente diseñada para posibilitar la experimentación en más o menos poco tiempo con redes de Aprendizaje Profundo. Sus fuertes se centran en ser amigable para el usuario, modular y extensible.

**Tensorflow**

Además trabajaremos con la libería **tensorflow**. TensorFlow es una biblioteca de código abierto para aprendizaje automático a través de un rango de tareas, y desarrollado por Google para satisfacer sus necesidades de sistemas capaces de construir y entrenar redes neuronales para detectar y descifrar patrones y correlaciones, análogos al aprendizaje y razonamiento usados por los humanos.

¡Hola! Estaré encantado de ayudarte a entender qué es un tokenizador y cómo puedes usar uno en tus proyectos de machine learning y procesamiento de lenguaje natural (NLP).

**¿Qué es un tokenizador?**

Recordando el concepto de tokenizador que realizamos en la práctica 2. Un tokenizador es una herramienta que se utiliza para dividir un texto en unidades más pequeñas llamadas tokens. Estos tokens suelen ser palabras, pero también pueden ser caracteres o subpalabras, dependiendo de cómo se configure el tokenizador. El proceso de tokenización es un paso fundamental en muchas tareas de NLP, ya que permite convertir texto no estructurado en una forma que los modelos de machine learning pueden entender y procesar.

Por ejemplo, la frase "Me gusta programar en Python" podría tokenizarse en los tokens ["Me", "gusta", "programar", "en", "Python"].

**Tokenizador de Keras**

El tokenizador `from keras.preprocessing.text import Tokenizer`, es una herramienta que viene con Keras. Este tokenizador realiza precisamente la tarea descrita anteriormente: divide el texto en tokens y puede convertir estos tokens en secuencias de números, lo que los hace útiles para entrenar modelos de deep learning.

Sin embargo, este componente ahora forma parte de TensorFlow, bajo `tf.keras.preprocessing.text.Tokenizer`. Desde que TensorFlow absorbió Keras como su API de alto nivel, la mayoría de las herramientas de Keras se acceden a través del módulo `tf.keras`.

**Tokenizador de TensorFlow**

El tokenizador que encontraste en TensorFlow (`tf.keras.preprocessing.text.Tokenizer`) funciona de manera muy similar al tokenizador original de Keras. Aquí tienes una descripción de los parámetros que mencionaste:

- `num_words`: El número máximo de palabras que se guardarán, basado en la frecuencia de palabra. Solo las `num_words` más comunes serán retenidas.
- `filters`: Una cadena en la que cada elemento es un carácter que se filtrará del texto. Es decir, estos caracteres serán ignorados durante la tokenización.
- `lower`: Un booleano que indica si se debe convertir el texto a minúsculas.
- `split`: El carácter que se utilizará como delimitador para dividir el texto en tokens.
- `char_level`: Si es True, cada carácter será tratado como un token.
- `oov_token`: Un valor que se asignará a los tokens fuera del vocabulario (palabras que no se han visto durante el entrenamiento).
- `analyzer`: Este parámetro no está documentado en la versión estándar de TensorFlow y podría no ser aplicable.

Para utilizar este tokenizador, primero instancias el objeto `Tokenizer` con los parámetros que desees y luego lo ajustas a tus textos utilizando el método `.fit_on_texts(texts)`. Después de ajustarlo, puedes usar `.texts_to_sequences(texts)` para convertir tus textos en secuencias de números, que son los índices de los tokens en el vocabulario del tokenizador.

In [10]:
import os
import re
import tensorflow as tf

In [7]:
# Instanciando el tokenizador
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=1000)

# Ejemplo de texto
texts = ["Me gusta programar en Python", "Python es genial para machine learning"]

# Ajustando el tokenizador en los textos
tokenizer.fit_on_texts(texts)

# Convirtiendo textos a secuencias de tokens
sequences = tokenizer.texts_to_sequences(texts)

print(sequences)

[[2, 3, 4, 5, 1], [1, 6, 7, 8, 9, 10]]


Ahora trabajaremos con los archivos del corpus MEX-A3T, necesitamos una función que lea los archivos del corpus y nos regrese el texto en una lista.

In [21]:
def get_text_from_file(path_corpus, path_truth):

    tr_text = []
    tr_y = []

    with open(path_corpus, "r") as f_corpus, open(path_truth, "r") as f_truth:
        for tweet in f_corpus:
            tr_text += [tweet]
        for label in f_truth:
            tr_y += [label]

    return tr_text, tr_y

In [22]:
path_text = "/Users/guillermo_sego/Desktop/Segundo Semestre/PLN/MexData/mex20_train.txt"
path_labels = "/Users/guillermo_sego/Desktop/Segundo Semestre/PLN/MexData/mex20_train_labels.txt"

tr_text, ty_y = get_text_from_file(path_text, path_labels)

In [25]:
tr_text[:5]

['@USUARIO @USUARIO @USUARIO Q se puede esperar del maricon de closet de la Yañez aun recuerdo esa ves q lo vi en zona rosa viendo quien lo levantada\n',
 '@USUARIO La piel nueva siempre arde un poquito los primeros días... y más con este puto clima\n',
 'Ustedes no se enamoran de mí… por tontas.\n',
 'Me las va a pagar esa puta gorda roba tuits...\n',
 '@USUARIO LA GENTE ES TONTA PORQUE NO SE DAN CUENTA QUE TÚ HACES A BATMAN AZUL\n']

## Estadísticas Simples

Vamos a realizar una inspección de los datos 